# TODO
- data preprocessing
    - 성별 미분류는 데이터에서 제거
- dummy variable
    - 모든 칼럼을 인코딩.
- pca.. or stepwise?
    - 수업에서 다룬 stepwise...
- visualization
    - not adequate methodology...
- how to make logistic regression...?
    - does logistic regression use gradient descent..?
    - no logistic regression is not necessary

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

import warnings

warnings.filterwarnings('ignore')

# korean font issue
plt.rcParams['font.family'] = 'AppleGothic'

In [ ]:
raw_data = pd.read_excel('./HMA_rawdata.xlsx', engine='openpyxl', sheet_name=['raw_data'])
raw_data = raw_data['raw_data']
raw_data.describe()

In [ ]:
# columns
DAY = '요일'
MEDIA = '발송매체'
IS_ADVERTISEMENT = '메시지 광고성 여부'
IS_INCLUDE_FINANCE = '메시지 금융 관련 내용 여부'
AGE = '연령대'
GENDER = '성별'
TIME = '발송시간대'
IS_SUCCESS = '발송성공여부'
LOGIN_DAYS_IN_MONTH = '최근 30일 간  로그인 일수'
IS_APPROVED = '메시지 수용 여부'

raw_data = raw_data[raw_data[GENDER] != '미분류'] # 성별 미분류 제외
raw_X = raw_data.drop(IS_APPROVED, axis=1)
raw_y = raw_data[IS_APPROVED]

In [ ]:
sns.histplot(raw_y)

In [ ]:
raw_data.describe()

In [ ]:
fig, ax = plt.subplots(3, 3)
fig.set_size_inches(20, 10)
x_list = [
    DAY,
    MEDIA,
    IS_ADVERTISEMENT,
    IS_INCLUDE_FINANCE,
    AGE,
    GENDER,
    TIME,
    IS_SUCCESS,
    LOGIN_DAYS_IN_MONTH,
]

for i in range(len(x_list)):
    row = i // 3 
    col = i % 3 
    sns.stripplot(x=x_list[i], y=IS_APPROVED, data=raw_data, ax=ax[row][col])

plt.show()

In [ ]:
columns = [
    DAY,
    MEDIA,
    IS_ADVERTISEMENT,
    IS_INCLUDE_FINANCE,
    AGE,
    GENDER,
    TIME,
    LOGIN_DAYS_IN_MONTH,
]

encoder = OneHotEncoder(drop='first', sparse=False)
X = encoder.fit_transform(raw_X.drop(IS_SUCCESS, axis=1))
X = sm.add_constant(X)

y = pd.get_dummies(raw_y, drop_first=True, dtype=int)

names = [''] + [*encoder.get_feature_names_out()]
tmp_X = X.copy()

model = sm.Logit(y, tmp_X).fit()

# Stepwise
while True:
    max_idx = np.argmax(model.pvalues)
    max_p_value = model.pvalues[max_idx]
    if max_p_value > 0.05:
        tmp_X = np.delete(tmp_X, max_idx, axis=1)
        name = names.pop(max_idx)
        model = sm.Logit(y, tmp_X).fit()
        print(model.summary())
        print(f">>> variable {max_idx}: {name} deleted")
    else:
        break


print(model.summary())
params = np.exp(model.params)
print('[beta에 exponential을 취했을 때의 값들]')
for i, param in enumerate(params):
    print(f'>>> {names[i] if i > 0 else "const"}: {param}')

In [ ]:
print(pd.crosstab(raw_data[DAY], raw_data[AGE]))
print('=========')
print(pd.crosstab(raw_data[MEDIA], raw_data[AGE]))
print('=========')
print(pd.crosstab(raw_data[IS_ADVERTISEMENT], raw_data[AGE]))
print('=========')
print(pd.crosstab(raw_data[IS_INCLUDE_FINANCE], raw_data[AGE]))
print('=========')
print(pd.crosstab(raw_data[GENDER], raw_data[AGE]))
print('=========')
print(pd.crosstab(raw_data[TIME], raw_data[AGE]))
print('=========')
print(pd.crosstab(raw_data[LOGIN_DAYS_IN_MONTH], raw_data[AGE]))
print('=========')
print(pd.crosstab(raw_data[IS_APPROVED], raw_data[AGE]))
print('=========')